## Importing Data and Libraries

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import (
    accuracy_score, 
    recall_score, 
    precision_score, 
    f1_score, 
    cohen_kappa_score
)
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC


In [10]:
Train = pd.read_csv('dataTrain.csv')
Test = pd.read_csv('dataTest.csv')

stats = pd.read_csv('stats.csv')
stats = stats.iloc[:,1:]

X_train = Train.drop('output', axis=1)
y_train = Train['output']

X_test = Test.drop('output', axis=1)
y_test = Test['output']

In [11]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.array
y_test = y_test.array

## Building the model

### RFC

In [12]:
# Define the Random Forest training function
def trainRandomForest(X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    accuracies, recalls, precisions, f1_scores, kappas = [], [], [], [], []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train the RandomForest model
        model = RandomForestClassifier(random_state=1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Compute and store metrics
        accuracies.append(accuracy_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
        kappas.append(cohen_kappa_score(y_test, y_pred))

    # Print average metrics
    print(f"Average Accuracy: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
    print(f"Average Recall: {np.mean(recalls):.3f} ± {np.std(recalls):.3f}")
    print(f"Average Precision: {np.mean(precisions):.3f} ± {np.std(precisions):.3f}")
    print(f"Average F1-Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
    print(f"Average Kappa: {np.mean(kappas):.3f} ± {np.std(kappas):.3f}")

# Example usage:
# Assuming X_train and y_train are NumPy arrays
trainRandomForest(X_train, y_train)


Average Accuracy: 0.822 ± 0.043
Average Recall: 0.822 ± 0.043
Average Precision: 0.826 ± 0.046
Average F1-Score: 0.820 ± 0.044
Average Kappa: 0.631 ± 0.087


XGB

In [13]:
# Define the Gradient Boosting training function
def trainGradientBoosting(X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    accuracies, recalls, precisions, f1_scores, kappas = [], [], [], [], []
    
    for train_index, test_index in kf.split(X):
        # Directly index the NumPy arrays
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Initialize XGBClassifier without use_label_encoder parameter
        model = XGBClassifier(eval_metric="logloss", random_state=1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Compute metrics and append to respective lists
        accuracies.append(accuracy_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
        kappas.append(cohen_kappa_score(y_test, y_pred))

    # Print average metrics
    print(f"Average Accuracy: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
    print(f"Average Recall: {np.mean(recalls):.3f} ± {np.std(recalls):.3f}")
    print(f"Average Precision: {np.mean(precisions):.3f} ± {np.std(precisions):.3f}")
    print(f"Average F1-Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
    print(f"Average Kappa: {np.mean(kappas):.3f} ± {np.std(kappas):.3f}")

# Example usage
# Assuming X_train and y_train are NumPy arrays
trainGradientBoosting(X_train, y_train)


Average Accuracy: 0.784 ± 0.041
Average Recall: 0.784 ± 0.041
Average Precision: 0.794 ± 0.050
Average F1-Score: 0.782 ± 0.041
Average Kappa: 0.553 ± 0.083


SVM

In [14]:
# Define the SVM training function
def trainSVM(X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    accuracies, recalls, precisions, f1_scores, kappas = [], [], [], [], []
    
    for train_index, test_index in kf.split(X):
        # Directly index the NumPy arrays
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Initialize the SVM model
        model = SVC(kernel='rbf', class_weight='balanced', random_state=1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Compute metrics and append to respective lists
        accuracies.append(accuracy_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
        kappas.append(cohen_kappa_score(y_test, y_pred))

    # Print average metrics
    print(f"Average Accuracy: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
    print(f"Average Recall: {np.mean(recalls):.3f} ± {np.std(recalls):.3f}")
    print(f"Average Precision: {np.mean(precisions):.3f} ± {np.std(precisions):.3f}")
    print(f"Average F1-Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
    print(f"Average Kappa: {np.mean(kappas):.3f} ± {np.std(kappas):.3f}")

# Example usage
# Assuming X_train and y_train are NumPy arrays
trainSVM(X_train, y_train)

Average Accuracy: 0.805 ± 0.037
Average Recall: 0.805 ± 0.037
Average Precision: 0.811 ± 0.039
Average F1-Score: 0.805 ± 0.036
Average Kappa: 0.601 ± 0.076
